In [1]:
import pandas as pd

# Cargar el archivo CSV con los datos filtrados y con Fold Change
file_path_fc = 'deSeqRes_filtrado_bacillus_subtilis_CW14_with_FC.csv'
data_with_fc = pd.read_csv(file_path_fc)

# Renombrar las columnas
data_with_fc = data_with_fc.rename(columns={
    'GeneSymbol': 'Genes',
    'DirectionalFoldChange': 'FoldChange'
})

# Crear una nueva columna 'Regulation' basada en el signo de 'FoldChange'
data_with_fc['Regulation'] = data_with_fc['FoldChange'].apply(lambda x: 'UpRegulated' if x > 0 else 'DownRegulated')

# Tomar el valor absoluto de 'FoldChange'
data_with_fc['FoldChange'] = data_with_fc['FoldChange'].abs()

# Cargar el archivo CSV con los términos GO
file_path_go = r'C:/Users/3060/OneDrive/Documentos/Tesis de Grado/Protocolo/4_Análisis de Enriquecimiento Funcional/Bacillus subtilis CW14 - Caco 2/Enrichr/Ontologies/GO_Biological_Process_2023_table.txt'
df = pd.read_csv(file_path_go, sep='\t')

# Eliminar las columnas especificadas
columns_to_drop = ["Overlap", "P-value", "Old P-value", "Old Adjusted P-value", "Odds Ratio", "Combined Score"]
df = df.drop(columns=columns_to_drop)

# Filtrar los registros con "Adjusted P-value" <= 0.05
df_filtered = df[df["Adjusted P-value"] <= 0.05]

# Expandir la columna "Genes" para que cada gen tenga su propia fila
df_expanded = df_filtered.assign(Genes=df_filtered['Genes'].str.split(';')).explode('Genes')

# Agrupar los términos asociados a cada gen en una lista
df_grouped = df_expanded.groupby('Genes').agg({
    'Term': lambda x: list(x),
    'Adjusted P-value': lambda x: list(x)  # Mantener todos los valores de "Adjusted P-value" para cada gen
}).reset_index()

# Combinar df_grouped con data_with_fc
df_combined = pd.merge(df_grouped, data_with_fc, on='Genes', how='left')

# Ordenar los registros por el valor de "FoldChange" del más alto al más bajo
df_combined_sorted = df_combined.sort_values(by='FoldChange', ascending=False)

# Convertir las columnas 'Term' y 'Adjusted P-value' a strings con los elementos de la lista separados por comas
df_combined_sorted['Term'] = df_combined_sorted['Term'].apply(lambda x: ', '.join(x))
df_combined_sorted['Adjusted P-value'] = df_combined_sorted['Adjusted P-value'].apply(lambda x: ', '.join(map(str, x)))

# Mostrar las primeras filas del dataframe combinado y ordenado
print(df_combined_sorted.head())

# Guardar el DataFrame combinado y ordenado en un nuevo archivo CSV
df_combined_sorted.to_csv('deSeqRes_combined_with_GO_Biological_Process_terms_sorted.csv', index=False)

# Guardar el DataFrame combinado y ordenado en un archivo Excel
df_combined_sorted.to_excel('deSeqRes_combined_with_GO_Biological_Process_terms_sorted.xlsx', index=False)

     Genes                                               Term  \
7     CCL4  Inflammatory Response (GO:0006954), Cytokine-M...   
12    CSF2  Cytokine-Mediated Signaling Pathway (GO:001922...   
13    CSF3  Cytokine-Mediated Signaling Pathway (GO:001922...   
21   CXCL8  Inflammatory Response (GO:0006954), Cytokine-M...   
16  CXCL10  Inflammatory Response (GO:0006954), Cytokine-M...   

                                     Adjusted P-value  Log2FoldChange  \
7   1.4828593548150188e-14, 3.3637027029829514e-14...        5.236074   
12  3.3637027029829514e-14, 1.166229904623436e-07,...        5.025177   
13  3.3637027029829514e-14, 1.970694749704463e-06,...        4.950078   
21  1.4828593548150188e-14, 3.3637027029829514e-14...        4.646226   
16  1.4828593548150188e-14, 3.3637027029829514e-14...        4.340264   

    FoldChange   Regulation  
7    37.689059  UpRegulated  
12   32.563351  UpRegulated  
13   30.911623  UpRegulated  
21   25.041102  UpRegulated  
16   20.255812  UpRe